## This notebook shows all the means to create and to configure DiagramDataset
* A `DiagramDataset` allows you to exploit information on the spatial position of different shots.
* A `DiagramDataset` allows you to manage data persistence in real time as well.

In [1]:
import shutil
import tempfile

from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc import Diagram, DiagramDataset

### How to add diagrams to the dataset ?
* All the means to init a `Diagram` are described in the `api_init_diagram` notebook.
* You can append diagrams before or after having define a function chain.

In [2]:
# from a pathlike
folder = get_samples()
dataset_1 = DiagramDataset(folder)  # pathlib.Path directory
dataset_2 = DiagramDataset(min(folder.glob("*.jp2")))  # pathlib.Path file

# from `Diagram` instances
diagrams = [Diagram(f) for f in sorted(folder.glob("*.jp2"))]
dataset_3 = DiagramDataset(diagrams)  # from an iterable of diagrams
dataset_4 = DiagramDataset(diagrams[0], diagrams[-1])  # from a few well-chosen diagrams

# big mix
dataset_5 = DiagramDataset(diagrams[0], diagrams[1], [diagrams[2], diagrams[3]], diagrams[4].file)

In [4]:
# from an uncomplet directory for real time
unfilled_folder = tempfile.mkdtemp()
print(unfilled_folder)

/tmp/tmppn9s9sf5


### How to apply a function ?